## imports

In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.utils import to_categorical
from  tensorflow.keras.preprocessing import text_dataset_from_directory
import numpy
from keras.preprocessing.sequence import pad_sequences
import os
import numpy
from keras.preprocessing.sequence import pad_sequences

# from tensorflow.keras.metrics import 


 ## methods to get data 

In [ ]:
def importData(directory: str) -> list:
    with open(directory) as f:
        return f.read()

def getAllFilesInFolder(dir: str) -> list:
    return os.listdir(dir)


## load all of the country music data into memory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
songs = []
for file in getAllFilesInFolder("/content/drive/MyDrive/song_data/Pop"):
    filePath = "/content/drive/MyDrive/song_data/Pop/{}".format(file)
    if "DS_Store" not in filePath:
        songs.append(importData(filePath))

len(songs)


100

## turn each word in text into integer-index representation 

In [ ]:

# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(songs)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# Transforms each text in texts to a sequence of integers.
encoded = tokenizer.texts_to_sequences(songs)
len(encoded)


Vocabulary Size: 2795


100

## get all of the triples and thier target word in the music data

In [ ]:

# create word -> word sequences 
# get sequences of words together, so given "a b c d e"
#  you should give [a,b,c,d], [b,c,d,e] adn all the adjacent triples and thier target word
sequences = []
for i in range(len(encoded)):
	for j in range(3, len(encoded[i])):
		sequence = encoded[i][j-3:j+1]
		sequences.append(sequence)
		
		
print('Total Sequences: %d' % len(sequences))
sequences

Total Sequences: 34564


[[36, 203, 325, 353],
 [203, 325, 353, 70],
 [325, 353, 70, 70],
 [353, 70, 70, 130],
 [70, 70, 130, 70],
 [70, 130, 70, 70],
 [130, 70, 70, 130],
 [70, 70, 130, 31],
 [70, 130, 31, 1],
 [130, 31, 1, 5],
 [31, 1, 5, 4],
 [1, 5, 4, 560],
 [5, 4, 560, 76],
 [4, 560, 76, 3],
 [560, 76, 3, 802],
 [76, 3, 802, 20],
 [3, 802, 20, 261],
 [802, 20, 261, 181],
 [20, 261, 181, 1505],
 [261, 181, 1505, 29],
 [181, 1505, 29, 127],
 [1505, 29, 127, 38],
 [29, 127, 38, 3],
 [127, 38, 3, 117],
 [38, 3, 117, 6],
 [3, 117, 6, 1506],
 [117, 6, 1506, 4],
 [6, 1506, 4, 10],
 [1506, 4, 10, 3],
 [4, 10, 3, 803],
 [10, 3, 803, 20],
 [3, 803, 20, 1507],
 [803, 20, 1507, 69],
 [20, 1507, 69, 3],
 [1507, 69, 3, 1508],
 [69, 3, 1508, 5],
 [3, 1508, 5, 3],
 [1508, 5, 3, 1158],
 [5, 3, 1158, 20],
 [3, 1158, 20, 3],
 [1158, 20, 3, 712],
 [20, 3, 712, 152],
 [3, 712, 152, 1],
 [712, 152, 1, 152],
 [152, 1, 152, 1],
 [1, 152, 1, 197],
 [152, 1, 197, 81],
 [1, 197, 81, 1],
 [197, 81, 1, 73],
 [81, 1, 73, 197],
 [1, 73

### make all senteances of equal length (pad them with space - keras req.)

In [ ]:
# pad sequences to equal lenth (Keras requirement)
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 4


## Get the word embeddings from GLOVE and add them to local memory

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/drive/MyDrive/song_data/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = numpy.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


## create weight matrix for glove embeddings

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = numpy.zeros((vocab_size, 100))
for word, i in  tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

### prepare data and labels for model 

In [ ]:
# convert to numpy and split into data (triples) and labels (target word)
sequences = numpy.array(sequences)
testData, testLabels = sequences[:,:-1],sequences[:,-1]

# one hot encode outputs -> Converts a class vector (integers) to binary class matrix.
testLabels= to_categorical(testLabels, num_classes=vocab_size)

### define and fit the model 

In [ ]:
# define model
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length-1, trainable=False)
model = Sequential()
model.add(e)
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(testData, testLabels, epochs=60, verbose=2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 100)            279500    
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 2795)              282295    
                                                                 
Total params: 642,195
Trainable params: 362,695
Non-trainable params: 279,500
_________________________________________________________________
None
Epoch 1/60
1081/1081 - 12s - loss: 6.0407 - accuracy: 0.0551 - 12s/epoch - 11ms/step
Epoch 2/60
1081/1081 - 6s - loss: 5.3835 - accuracy: 0.0948 - 6s/epoch - 6ms/step
Epoch 3/60
1081/1081 - 6s - loss: 4.8996 - accuracy: 0.1376 - 6s/epoch - 6ms/step
Epoch 4/60
1081/1081 - 6s - loss: 4.4945 - accuracy: 0

### method to generate sequences of text from model

In [ ]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = numpy.argmax(model.predict(encoded), axis=-1)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [ ]:
import random

def getSentanceStarter(sentanceStarterLenth, songs):
    randomSongUpperBound = len(songs)
    #
    randomSongIndex = random.randint(0, randomSongUpperBound-1)
    randomSong = songs[randomSongIndex]
    splitSong = randomSong.split("\n")
    randomLineIndex = random.randint(0, len(splitSong)-1)
    line = splitSong[randomLineIndex]
    splitLine = line.split(" ")
    joinedSelection = " ".join(splitLine[0:sentanceStarterLenth])
    return joinedSelection



In [ ]:
for i in range(10):
    print(generate_seq(model, tokenizer, max_length-1, getSentanceStarter(3, songs), 10))

Run away, now run away now run away now run away now run
And then we start to dance and now i'm all up on ya
 chennu chennu chennu chennu chennu chennu chennu chennu chennu chennu
Ooh ooh oooh ooh ooohooh so sweet so tight so tight i won't
And you give me all of you love your curves and all your
Lightning, lightning, lightning lightning lightning lightning lightning lightning lightning lightning lightning lightning lightning
Start a mob and you can keep me inside the pocket of your
And I will always love you i will always love you i will
 chennu chennu chennu chennu chennu chennu chennu chennu chennu chennu
I don't care what happened but i ain't feel my face but i


In [ ]:
# print(generate_seq(model, tokenizer, "I", 6))
print(generate_seq(model, tokenizer, max_length-1, "I can see", 100))

I can see it in your eyes of blue that is why all the girls in town follow you all around just like me they long to be close to you on the day that you were born the angels got together and decided to create a dream come true so they sprinkled moon dust in your hair of gold and star light in your eyes of blue that is why all the girls in town follow you all around just like me they long to be close to you on the day that you were born the angels got together and decided


## eval with BLEU score
https://towardsdatascience.com/how-to-evaluate-texxt-generation-models-metrics-for-automatic-evaluation-of-nlp-models-e1c251b04ec1


In [ ]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu


def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

In [ ]:
bleu(["It's hittin' rock bottom smoke 'em if you got 'em"],["nothing's going right makin' i just wish you were a"])

0.025584278811044955